 <h2 style="text-align: center;">НАЦИОНАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ УНИВЕРСИТЕТ 
 <br/>МОСКОВСКИЙ ЭНЕРГЕТИЧЕСКИЙ ИНСТИТУТ</h2>
 </br>
 <h3 style="text-align: center;">Кафедра Паровых и Газовых Турбин</h3>

 </br>
 </br>
 </br>
 </br>
 </br>

 <h1 style="text-align: center;">Курсовая работа по курсу:</h1>
 <h1 style="text-align: center;">«Паровые и газовые турбины»</h1>

 </br>
 </br>
 </br>
 </br>
 </br>

 <h5 style="text-align: right;">Студент: Иванов Иван Иванович</h5>
 <h5 style="text-align: right;">Группа: С-04-15</h5>

 </br>
 </br>
 </br>
 </br>
 </br>
 </br>
 </br>
 </br>
 </br>
 </br>
 </br>
 </br>
 </br>
 </br>
 </br>

 <h5 style="text-align: center;">Москва 2023</h5>



 ## Вариант №1

 ### $\space$ Дано:
 * $P_0 = 24.5 \space МПа $ - <Давление острого пара>
 * $t_0 = 560 ^\circ C$ -  <Температура острого пара>
 * $P_{пп} = 3.46 \space МПа$ -  <Давление промежуточного перегрева>
 * $t_{пп} = 565 ^\circ C$ -  <Температура горячей нитки промежуточного перегрева>
 * $P_{к} = 3.7 \space кПа$ -  <Давление в конденсаторе>
 * $t_{п.в.} = 268^\circ C$ -  <Температура питательной воды>

 * $N_{э} = 310 \space МВт$ -  <Электрическая мощность>
 * $z = 8$ -  <Количество регенеративных подогревателей>


In [9]:
import math
import iapws
import numpy
import matplotlib.pyplot as plt
from iapws import IAPWS97 as gas
from math import sqrt
from enum import Enum
import sys
import dataclasses

In [10]:
sys.path.append("..")
from Steam_Turbine_Machine import STM, plot_hs_diagram, plot_process
from Turbine_Stage import Turbine_Stage
from Array_Data import Array_Data
from Vector_2 import vector2

In [11]:
MPa = 10 ** 6
kPa = 10 ** 3
unit = 1 / MPa
cm = 1/100
mm = 1 / 1000
kJ = 1000
to_kelvin = lambda x: x + 273.15 if x else None

In [12]:
electrical_power = 310 * (10 ** 6)
p0 = 24.5 * MPa
t0 = 560
p_middle = 3.46 * MPa
t_middle = 565
pk = 3.7 * kPa
t_feed_water = 268
p_feed_water = 1.4 * p0
z = 8

internal_efficiency = 0.85
mechanical_efficiency = 0.994
generator_efficiency = 0.99

In [13]:
turbine = STM(electrical_power, p0, t0, p_middle, t_middle, pk, t_feed_water, 
              p_feed_water, z, internal_efficiency, mechanical_efficiency, generator_efficiency)
print("Массовый расход в турбину на входе: ", turbine.inlet_mass_flow, 
      "\nМассовый расход в конденсатор:", turbine.condenser_mass_flow)

Массовый расход в турбину на входе:  239.12057265187462 
Массовый расход в конденсатор: 159.54364436417112


In [ ]:
fig, ax  = plt.subplots(1, 1, figsize=(15, 15))
plot_hs_diagram(
    ax,
    points=[turbine._point_0, turbine.point_0, turbine.point_1t, turbine.point_1, 
            turbine._point_middle, turbine.point_middle, turbine.point_2, turbine.point_2t]
)
plot_process(ax, points=[turbine._point_0, turbine.point_0, turbine.point_1], color='black')
plot_process(ax, points=[turbine._point_middle, turbine.point_middle, turbine.point_2], color='black')
plot_process(ax, points=[turbine._point_0, turbine.point_0, turbine.point_1t], alpha=0.5, color='grey')
plot_process(ax, points=[turbine._point_middle, turbine.point_middle, turbine.point_2t], alpha=0.5, color='grey')

In [ ]:
G = turbine.inlet_mass_flow
d = 1.08
n = 60
P0 = p0
t0 = t0
c0 = 0
H0 = numpy.array([90,110])
p  = 0.1
a0 = 90

P0, t0, G

stages = [Turbine_Stage(G, p0, t0, c0, i, p, d, n, a0) for i in range(H0[0], H0[1]+1)]

#stage_variant.alpha1_e, stage_variant.alpha0

In [ ]:
for stage_variant in stages:
    stage_variant.set_nozzle_array('c_90_15_a', 0.75, stage_variant.el1 / stage_variant.e_opt, stage_variant.c1_t / stage_variant.point_1t.w)

In [ ]:
for stage_variant in stages:
    stage_variant.nozzle_array.set_specified_params(38, 0.065, stage_variant.alpha1_e, stage_variant.alpha0)
    stage_variant.nozzle_array.check_for_errors()

In [ ]:
for stage_variant in stages:
    stage_variant.calc_point1()
#stage_variant.w1, stage_variant.c1, stage_variant.betta1, stage_variant.w2t

In [ ]:
for stage_variant in stages:
    stage_variant.set_working_array('r_30_21_a', 0.6, stage_variant.nozzle_array.l + stage_variant.delta, stage_variant.w2t / stage_variant.point_2t.w)
#stage_variant.working_array.t, stage_variant.working_array.M

In [ ]:
for stage_variant in stages:
    stage_variant.working_array.set_specified_params(81, 0.105, math.degrees(math.sin(stage_variant.F2 / stage_variant.e_opt / math.pi / stage_variant.d / stage_variant.working_array.l)), stage_variant.betta1)
    stage_variant.working_array.check_for_errors()

In [ ]:
for stage_variant in stages:
    stage_variant.calc_point_2()
    stage_variant.calc_efficiency()
#stage_variant.N_i, stage_variant.etta_oi

In [ ]:
for stage_variant in stages:
    stage_variant.calc_durability()

In [ ]:
plt.style.use('_mpl-gallery')

fig, ax = plt.subplots(figsize=(20, 10.8), layout='constrained')

ax.plot([k.H0 for k in stages], [k.etta_oi for k in stages], linewidth=1, color = "red")

# make data

# plot
ax.set(xlim=(90, 110), xticks=numpy.arange(90, 110, 5),
       ylim=(0.6, 0.8), yticks=numpy.arange(0.6, 0.8, 0.01))

plt.show()

In [ ]:
optimal_stage = max(stages, key = lambda x : x.etta_oi)
print(optimal_stage.H0, optimal_stage.etta_oi)

w1 = vector2(-optimal_stage.w1 * math.cos(math.radians(optimal_stage.betta1)), -optimal_stage.w1 * math.sin(math.radians(optimal_stage.betta1)))
c1 = vector2(-optimal_stage.c1 * math.cos(math.radians(optimal_stage.alpha1)), -optimal_stage.c1 * math.sin(math.radians(optimal_stage.alpha1)))    
w2 = vector2(optimal_stage.w2 * math.cos(math.radians(optimal_stage.betta2)), -optimal_stage.w2 * math.sin(math.radians(optimal_stage.betta2)))
c2 = vector2(optimal_stage.c2 * math.cos(math.radians(optimal_stage.alpha2)), -optimal_stage.c2 * math.sin(math.radians(optimal_stage.alpha2)))    

plt.style.use('_mpl-gallery')

fig, ax = plt.subplots(figsize=(20, 10.8), layout='constrained')

ax.plot([w1.x0, w1.x1], [w1.y0, w1.y1], linewidth=1, color = "blue")
ax.plot([w2.x0, w2.x1], [w2.y0, w2.y1], linewidth=1, color = "blue")
ax.plot([c1.x0, c1.x1], [c1.y0, c1.y1], linewidth=1, color = "red")
ax.plot([c2.x0, c2.x1], [c2.y0, c2.y1], linewidth=1, color = "red")

ax.plot([w1.x1, c1.x1], [w1.y1, c1.y1], linewidth=1, color = "black")
ax.plot([w2.x1, c2.x1], [w2.y1, c2.y1], linewidth=1, color = "black")


# make data

# plot
ax.set(xlim=(-400, 250), xticks=numpy.arange(-400, 250, 50),
       ylim=(-150, 50), yticks=numpy.arange(-150, 50, 50))

plt.show()